# Examples & Exercises from course literature

## Examples from chapter 1 - Exploratory Data Analysis with MATLAB

In [1]:
import scipy.io
import pandas as pd
import numpy as np
import plotly.express as px
import os

### Example 1.1 

In [2]:
leukemia_matlab_data_path = os.path.join(os.pardir, 'data', 'leukemia.mat')
leukemia_mat = scipy.io.loadmat(leukemia_matlab_data_path)

In [3]:
leukemia_sample = leukemia_mat['leukemia'][:, :38]
leukemia_sample

array([[ 3105,  1118,  4543, ...,   473,   186,   680],
       [ 3016,  3424,  7724, ...,  1648,   528,    99],
       [ 9326,   895,   628, ...,   -14,   218,   251],
       ...,
       [  382,  3606,  2997, ..., 15519, 13221,  2702],
       [  -70,   364,  -208, ...,  2892,   157,  -245],
       [ 1057,  1395,   823, ...,  5162,  2887,  4681]], dtype=int16)

In [4]:
def standardize(row):
    return (row - np.mean(row))/np.std(row)

standardized_leukemia_sample = np.apply_along_axis(standardize, 1, leukemia_sample)
standardized_leukemia_sample

array([[ 7.13392909e-02, -8.33202186e-01,  7.25959645e-01, ...,
        -1.12682536e+00, -1.25747630e+00, -1.03259281e+00],
       [-1.70982956e-01, -1.80050978e-03,  1.78124979e+00, ...,
        -7.38241748e-01, -1.20266415e+00, -1.38055452e+00],
       [ 2.48265115e+00, -6.36280421e-01, -7.35053370e-01, ...,
        -9.72552370e-01, -8.86727186e-01, -8.74519293e-01],
       ...,
       [-7.63574528e-01,  1.15916306e-01, -5.02158024e-02, ...,
         3.36572223e+00,  2.73883950e+00, -1.30690305e-01],
       [-5.77803278e-01,  8.15153175e-02, -7.87448362e-01, ...,
         3.92197018e+00, -2.32952308e-01, -8.43657551e-01],
       [-5.99778532e-01, -4.27157575e-01, -7.19285349e-01, ...,
         1.49669789e+00,  3.34826060e-01,  1.25104499e+00]])

In [5]:
fig = px.imshow(standardized_leukemia_sample, origin='lower')
fig.update_layout(
    width=600,
    height=600,
    title='Gene Expression for Leukemia'
    )
fig.update_xaxes(title='ALL (1-27) or AML (28-38)')
fig.update_yaxes(title='Gene')
fig.show()

### Example 1.2

Log transformation on software inspection data 

In [6]:
software_matlab_data_path = os.path.join(os.pardir, 'data', 'software.mat')
software_mat = scipy.io.loadmat(software_matlab_data_path)
software_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'prepsloc', 'defsloc', 'mtgsloc', 'prepage', 'defpage'])

In [7]:
prepsloc = np.log(software_mat['prepsloc']).reshape(-1)
defsloc = np.log(software_mat['defsloc']).reshape(-1)


fig = px.scatter(x=prepsloc, y=defsloc)
fig.update_xaxes(title='Log PrepTime/SLOC)')
fig.update_yaxes(title='Log Defects/SLOC')
fig.show()

### Example 1.3
Sphering the Data - we generate 2–D multivariate normal random variables that have the following parameters: $$ \mu = \begin{bmatrix} -2 \\ 2 \end{bmatrix}, $$ and $$ \Sigma = \begin{bmatrix} 1 & 0.5 \\ 0.5 & 1 \end{bmatrix}$$ where Σ is the covariance matrix


In [8]:
number_samples = 100
mu = [-2, 2]
sigma = [[1,.5],[.5,1]]
samples = np.random.multivariate_normal(mu, sigma, number_samples)

px.scatter(x=samples[:, 0], y=samples[:, 1])


In [9]:
x_bar = np.mean(samples, axis=0) # axis=0 is important since we take the means of the columns
centered_samples = samples - x_bar
transposed_centered_samples = centered_samples.T

px.scatter(x=centered_samples[:, 0], y=centered_samples[:, 1])

In [10]:
covariance_matrix = np.cov(transposed_centered_samples, rowvar=True, bias=True)
eigen_values, eigen_vectors = np.linalg.eig(covariance_matrix)

In [11]:
# Calculate inverse square root of Eigenvalues
inv_root_eigen_values =  np.diag(1/eigen_values**0.5) 
real_inv_root_eigen_values = inv_root_eigen_values.real.round(4)
real_inv_root_eigen_values

array([[1.4159, 0.    ],
       [0.    , 0.7798]])

In [12]:
sphered_data = np.dot(
                        np.dot(real_inv_root_eigen_values, eigen_vectors.T),
                        transposed_centered_samples
                    )

In [13]:
fig = px.scatter(x=sphered_data.real[0, :], y=sphered_data.real[1, :])
fig.show()

## Exercises from Chapter 1

### Exercise 1.2 

Repeat Example 1.1 using the remaining columns (39 – 72) of the
leukemia data set. Does this follow the same pattern as the others?

In [14]:
leukemia_sample = leukemia_mat['leukemia'][:, 39:]
standardized_leukemia_sample = np.apply_along_axis(standardize, 1, leukemia_sample)
fig = px.imshow(standardized_leukemia_sample, origin='lower')
fig.update_layout(
    width=600,
    height=600,
    title='Gene Expression for Leukemia'
    )
fig.update_yaxes(title='Gene')
fig.show()


### Exercise 1.3

Repeat Example 1.1 using the lungB gene expression data set. Is there
a pattern? (looking at only first 50 genes)

In [15]:
lung_b_matlab_data_path = os.path.join(os.pardir, 'data', 'lungB.mat')
lung_b_mat = scipy.io.loadmat(lung_b_matlab_data_path)
lung_b_sample = lung_b_mat['lungB']
lung_b_sample.shape

(675, 156)

In [16]:
standardized_lung_b_sample = np.apply_along_axis(standardize, 1, lung_b_sample[:50, :-17])
fig = px.imshow(standardized_lung_b_sample, origin='lower')
fig.update_layout(

    title='Gene Expression for Lung Cancer'
    )
fig.update_yaxes(title='Gene')
fig.show()

### Exercise 1.8 

Generate n = 2 normally distributed random variables.

Find the Euclidean distance between the points after they have been transformed first using Equation 1.1 and then Equation 1.2.

Are the distances the same?

In [17]:
samples = np.random.normal(size=2)

In [18]:
# Applying eq 1.1: z-score 
z_scored_samples = (samples - np.mean(samples))/np.std(samples)
euclidean_dist_z_socred_samples = np.linalg.norm(z_scored_samples[0] - z_scored_samples[1])

In [19]:
samples_over_std = samples/np.std(samples)
euclidean_dist_samples_over_std = np.linalg.norm(samples_over_std[0] - samples_over_std[1])

In [20]:
euclidean_dist_z_socred_samples == euclidean_dist_samples_over_std

True

## Examples from chapter 2

### Example 2.1 - Projection 

Assume the data set consists of the two bivariate points 

$$ x_1 = \begin{bmatrix} 4 \\ 3 \end{bmatrix} \;\;\; x_2 = \begin{bmatrix} -4 \\ 5 \end{bmatrix} $$

and we are projecting onto a line that is $\theta$ radians from the horizontal or $x$-axis. For this example, the projection matrix is given by

$$ \text{P} =  \begin{bmatrix} (\text{cos}\:\theta)^2 & \text{cos}\:\theta\:\text{sin}\:\theta\\  \text{cos}\:\theta\:\text{sin}\:\theta & (\text{sin}\:\theta)^2\end{bmatrix} $$

In [21]:
x = np.array([[4, 3], [-4, 5]])
theta = np.pi / 3
cos_pow_2 = np.cos(theta)**2
cos_sin = np.cos(theta)*np.sin(theta)
sin_pow_2 = np.sin(theta)**2
p = np.array([[cos_pow_2, cos_sin], [cos_sin, sin_pow_2]])

In [22]:
px.scatter(x=x[1, :], y=x[0, :])

In [23]:
projection = np.dot(x,p)
px.scatter(x=projection[1, :], y=projection[0, :])

In [24]:
# projecting onto 1-D space 

horizontal_projection_matrix = np.array([[1], [0]])
one_d_projection = np.dot(x, horizontal_projection_matrix)
px.scatter(x=one_d_projection.squeeze(), y=np.zeros_like(one_d_projection).squeeze())

### Example 2.2 - PCA

In [25]:
yeast_data_pth = os.path.join(os.pardir, 'data', 'yeast.mat')
yeast_mat = scipy.io.loadmat(yeast_data_pth)
yeast_gene_data = yeast_mat['data']

In [26]:
print(f'Data is a matrix of shape {yeast_gene_data.shape}')

def center(row):
    return row - np.mean(row)

centered_yeast = np.apply_along_axis(lambda x: x-x.mean(), 0, yeast_gene_data)
cov_yeast = np.cov(centered_yeast.T)
eigen_values, eigen_vectors =  np.linalg.eig(cov_yeast)


Data is a matrix of shape (384, 17)


In [27]:
fig = px.line(y=eigen_values)
fig.update_layout(

    title='Scree plot'
    )
fig.update_yaxes(title='Eigenvalue')
fig.update_xaxes(title='Eigenvalue Index')

In [28]:
# the percentage of variance explained
var_explained = 100*np.cumsum(eigen_values)/np.sum(eigen_values)
var_explained

array([ 73.59226325,  85.0875057 ,  91.96561088,  94.32169204,
        95.56162686,  96.49464828,  97.36796926,  98.02585856,
        98.46994366,  98.87426452,  99.17311025,  99.42323453,
        99.5917227 ,  99.74383868,  99.81163792,  99.90328488,
       100.        ])

Using 95% as a threshold we would select the first 4/5 PCs 

In [49]:
# using broken stick method
p = yeast_gene_data.shape[1]
expected_size = np.zeros((p))
for k in range(1, p+1):
    for i in range(k, p+1):
        expected_size[k-1] += 1/i
expected_size = expected_size/p


In [50]:
prop_variance = eigen_values/np.sum(eigen_values)
print(expected_size[:4])
print(prop_variance[:4])

[0.20232662 0.14350309 0.11409132 0.09448348]
[0.73592263 0.11495242 0.06878105 0.02356081]


Based on the Broken stick method we will only keep the first PC


Selection of PCs using the size of the variance 

In [54]:
avg_eigen_val = np.mean(eigen_values)
num_pcs_to_kep = len([x for x in eigen_values if x > avg_eigen_val])
print(f'we will keep {num_pcs_to_kep} dimensions')

we will keep 3 dimensions


In [65]:
pc_vectors = eigen_vectors[:, :3]
pc_scores = np.dot(centered_yeast, pc_vectors)
pc_scores.shape

(384, 3)

In [79]:
fig = px.scatter_3d(x=pc_scores[:,0], y=pc_scores[:,1], z=pc_scores[:,2], mode)
fig.update_layout(
    title='Yeast data projected onto first 3 PCs',
    scene = dict(
                    xaxis_title='PC 1',
                    yaxis_title='PC 2',
                    zaxis_title='PC 3'),
                    margin=dict(r=20, b=10, l=10, t=10)
    )
fig.show()